###  **Büyük Veri Setleri Üzerinde Veri Madenciliği Dersi - Aktivite2:** Birliktelik kuralı çıkarma ve bazı parametreleri anlama
#### **Author:** Süheyl Çavuşoğlu
**Problem:**
- 1- Ekte verilen “store_data.csv” dataset icinde yapılan alıs-verislere ait transactionlar var. İlk satır I(tüm itemlara) gösterir.
- 2- İstediğiniz bir platformda Apriori algoritması icin top-K rule donen kod yazınız. K, kod icinde hard-coded veya kullanıcı belirlemesi şeklinde olabilir.
- 3- Bulunan kuralların önemi (sıralaması) bazı istatistiki ölçütlere göre yapılır. 2'ye benzer sekilde aşağıdaki ölçütleri hard-coded veya kullanıcıdan alan şekilde organize ederek hangi ölçüte göre top-K nasıl değisiyor inceleyiniz.
    - _support_ - _confidence_ - _cosine_ - _Gini_
- 4- min support değerini çok düşük veya çok yüksek tuttuğunuzda elde ettiğiniz kuralları inceleyip yorumlayınız. Yorumlarınızı içeren dökümanı ve kodları edestek sistemine yükleyiniz.

In [ ]:
!pip install mlxtend  # İlk olarak Apriori algoritması için kullanacağımız "mlxtend (Machine Learning eXtensions)" kütüphanesini kurduk.

In [1]:
import pandas as pd
df = pd.read_csv("store_data.csv", header=None)  # stora_data.csv isimli veri setimizi Pandas kütüphanesi yardımı ile import ettik.
df.head(10)  # verisetimizin ilk 10 satırını gözlemledik.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape

(7501, 20)

In [2]:
# veri setimizi Pandas dataframe'i şeklinde tuttuk ancak bu format Apriori algoritması kullanarak birliktelik kuralları çıkarmak için uygun değil. dolayısıla veri setimizin formatını uygun hale  getirmeliyiz.

import numpy as np
from mlxtend.preprocessing import TransactionEncoder  #mlxtend kütüphanesi içerisinden TransactionEncoder sınıfını kullanarak verisetimizin içerisindeki eşyaları sıklıklarına göre belirleyebiliyoruz.
from mlxtend.frequent_patterns import apriori, association_rules  # apriori algoritması kullanarak birliktelik kuralları çıkarmak için de mlxtend kütüphanesini kullanacağız.

# aşağıda transactions listesinin içerisine her bir gözlemdeki eşyaları sırayla ekledik.
transactions = []
for i in range(df.shape[0]): # satır sayısı kadar bir range belirledik ki bunlar gözlem sayımız olmuş oluyor ve for loop'umuzu burada döndürdük.
    transactions.append([str(df.values[i, j]) for j in range(df.shape[1]) if str(df.values[i, j]) != 'nan']) # burada nan gördüğü değerleri listeye almaması için filtre uyguladık.

trans_enc = TransactionEncoder()
trans_enc_array = trans_enc.fit(transactions).transform(transactions)  # transaction encoder sınıfı ile birlikte alınan ürünlerin sıklığını belirledik. bu sınıf içerisinden fit metodu ile sıklık matrisi oluşturduk ve bu matrisi transform metodu ile bir numpy dizisinde sakladık.
df_encoded = pd.DataFrame(trans_enc_array, columns=trans_enc.columns_) # hazırladığımız numpy dizisini ve sütun isimleri olarak kodlanan, verisetimizin içerisindeki ürünleri kullanarak bir dataframe oluşturduk. bu dataframe 'i apriori fonksiyonu için kullanacağız.

#birliktelik kuralı için en iyi k'yı belirleyen fonksiyonu yazacağız. bu fonksiyon döndürülecek en iyi kural sayısı olan k'yı, minimum destek sayısı olan min_support değerini ve hesaplama için kullanılacak ölçüt olan metric'i parametre olarak alıyor. 
# ascending = False diyerek de sonuçları yukarıdan aşağıya azalan şekilde gösteriyoruz.
def get_top_k_rules(k, min_support=0.01, metric='confidence', ascending=False):
    
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True) # en sık geçen itemsetleri belirledik.
    
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=0.1) # birliktelik kurallarını hesapladık.
    
    return rules.sort_values(by=metric, ascending=ascending).head(k) # dataframe'imiz içerisinde ölçüte göre yukarıdan aşağıya azalan şekilde k değerlerini döndürdük.

top_k_rules = get_top_k_rules(5) # parantez içerisine yazılan (5) k sayısına göre en iyi kuralları yazdırdık.

In [22]:
top_k_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
280,"(eggs, ground beef)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,0.005365,1.543848
302,"(milk, ground beef)",(mineral water),0.021997,0.238368,0.011065,0.503030,2.110308,0.005822,1.532552
266,"(chocolate, ground beef)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,0.005434,1.447937
294,"(milk, frozen vegetables)",(mineral water),0.023597,0.238368,0.011065,0.468927,1.967236,0.005440,1.434136
232,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,0.011020,1.401255


In [3]:
# şimdi de support, confidence, cosine ve Gini istatistiksel ölçütlerine göre top-K kuralları nasıl değişiyor, bunu inceleyelim.

from scipy.spatial.distance import cosine # cosine istatistiğini incelemek için scipy.spatial.distance içerisinden cosine fonksiyonunu import ettik.

def gini_coefficient(confidence, support):
    return (confidence * (1 - support)) / (confidence + support)  # bir kuralın confidence ve support değerlerine göre Gini katsayısını hesaplamak için bir fonksiyon yazıyoruz.

def calculate_additional_metrics(rules): # oluşturduğumuz yeni dataframe'i parametre olarak alan bir fonksiyon yazıyoruz. bu fonksiyon her bir kural için cosine ve gini istatistiklerini hesaplıyor.
    # Cosine
    rules['cosine'] = rules.apply(lambda row: 1 - cosine([row['support'], row['confidence']], [0, 0]), axis=1) # cosine istatistiğinı hesapladık ve dataframe'imizin cosine kolonuna bu sonucu ekledik.
    
    # Gini
    rules['gini'] = rules.apply(lambda row: gini_coefficient(row['confidence'], row['support']), axis=1) # gini istatistiğini hesapladık ve dataframe'imizin gibi kolonuna bu sonucu ekledik.
    
    return rules # dataframe'imizin son halini döndürüyoruz.

def compare_top_k_rules(k, min_support=0.01): # min_support değerimizi default olarak 0.01 aldık, farklı min_support değerlerini incelemek istersek değiştirebileceğiz.
    metrics = ['support', 'confidence', 'cosine', 'gini']
    top_k_rules_dict = {}
    
    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True) # verisetimizde en sık geçen eşyaları hesapladık.
    
    rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1) # birliktelik kurallarını hesapladıkk.

    rules = calculate_additional_metrics(rules)  # cosine ve gini istatistiklerini hesapladık.

    for metric in metrics: # inceleyeceğimiz tüm ölçütler için bir for loop yazdık. bu loop metrics listesine yazdığımız tüm ölçütler için dönüyor ve her ölçüt için ayrı ayrı top-k kurallarını bulup yazdırıyor.
        top_k_rules_dict[metric] = rules.nlargest(k, metric)
        print(f"Top {k} Rules by '{metric}':")
        print(top_k_rules_dict[metric][['antecedents', 'consequents', metric]])
        print()

compare_top_k_rules(5) # parantez içerisine yazılan (5) k sayısına göre tüm ölçütlerimiz için yapılan hesaplamaları yazdırıyoruz.
# burada min_support değerini düşük (0.01) olarak aldık ve hesaplamaları ona göre yaptırdık. 

Top 5 Rules by 'support':
         antecedents      consequents   support
233  (mineral water)      (spaghetti)  0.059725
234      (spaghetti)  (mineral water)  0.059725
71   (mineral water)      (chocolate)  0.052660
72       (chocolate)  (mineral water)  0.052660
107           (eggs)  (mineral water)  0.050927

Top 5 Rules by 'confidence':
                   antecedents      consequents  confidence
279        (eggs, ground beef)  (mineral water)    0.506667
302        (milk, ground beef)  (mineral water)    0.503030
266   (chocolate, ground beef)  (mineral water)    0.473988
294  (milk, frozen vegetables)  (mineral water)    0.468927
232                     (soup)  (mineral water)    0.456464

Top 5 Rules by 'cosine':
   antecedents      consequents  cosine
0    (avocado)  (mineral water)       1
1    (burgers)           (cake)       1
2       (cake)        (burgers)       1
3    (burgers)      (chocolate)       1
4  (chocolate)        (burgers)       1

Top 5 Rules by 'gini':
      

c:\Users\suheyl\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [4]:
# şimdi de min_support değerini daha yüksek (0.05) alalım ve sonuçları kontrol edelim.
print("Min Support: 0.05")
compare_top_k_rules(5, min_support=0.05)

Min Support: 0.05
Top 5 Rules by 'support':
       antecedents      consequents   support
4  (mineral water)      (spaghetti)  0.059725
5      (spaghetti)  (mineral water)  0.059725
0  (mineral water)      (chocolate)  0.052660
1      (chocolate)  (mineral water)  0.052660
2           (eggs)  (mineral water)  0.050927

Top 5 Rules by 'confidence':
       antecedents      consequents  confidence
5      (spaghetti)  (mineral water)    0.343032
1      (chocolate)  (mineral water)    0.321400
2           (eggs)  (mineral water)    0.283383
4  (mineral water)      (spaghetti)    0.250559
0  (mineral water)      (chocolate)    0.220917

Top 5 Rules by 'cosine':
       antecedents      consequents  cosine
0  (mineral water)      (chocolate)       1
1      (chocolate)  (mineral water)       1
2           (eggs)  (mineral water)       1
3  (mineral water)           (eggs)       1
4  (mineral water)      (spaghetti)       1

Top 5 Rules by 'gini':
       antecedents      consequents      gini
1 

c:\Users\suheyl\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\spatial\distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Son olarak min support değerini çok düşük (0.01) ve çok yüksek (0.05) tuttuğumuzda elde ettiğimiz çıktıları, kuralları inceleyip yorumlayalım.


- **Support:** İki çıktının da 'support' ölçütü için sonuçlarının aynı olduğunu gördük. Hem min_support değeri 0.01 olan çıktıda hem de min_support değeri 0.05 olan çıktıda en sık görülen kuralın mineral water ve spaghetti'nin birlikte alındığı olduğunu gördük.

- **Confidence:** Min_support 0.01 olan çıktıda, confidence değerleri daha yüksek olan karmaşık kurallar gözlemleniyor. Min_support 0.05 olan çıktıda ise, confidence değerleri daha düşük ve daha basit kurallar elde ediliyor.

- **Cosine:** Min_support 0.01 olan çıktıda, ürün kombinasyonları arasındaki bağımsızlık farklıdır. Min_support 0.05 olan çıktıda ise, sadece mineral water'ın diğer ürünlerle olan bağımsızlık ilişkisi gösterilir.

- **Gini:** Min_support 0.01 olan çıktıda, Gini değerleri daha yüksek ve daha karmaşık kurallar gözlemlenir. Min_support 0.05 olan çıktıda ise, Gini değerleri daha düşük ve daha basit kurallar elde edilir.

Çok düşük min_support değeri (0.01) kullanıldığında, daha karmaşık ve özelleştirilmiş ilişkileri incelemeye olanak sağlar; ancak, bazen daha az anlamlı veya gürültülü kuralların da ortaya çıkmasına neden olabilir. Çok yüksek min_support değeri (0.05) kullanıldığında ise, daha genel eğilimler ve ilişkiler hakkında bilgi edinmeye yardımcı olabilir; fakat, daha seyrek görülen ancak potansiyel olarak değerli ilişkilerin ve eğilimlerin göz ardı edilmesine neden olabilir.

İdeal min_support değerini belirlemek, ihtiyaçlarımıza ve analiz amacımıza bağlıdır. En uygun min_support değerini ise deneme yanılma yolu ile saptayabiliriz.